# Automatic weather station examples 
Eric Gagliano (egagli@uw.edu)   
Updated: March 7th, 2024

**Thanks for checking out these examples! The automatic_weather_station module is intended to make it easier to retrieve daily SNOTEL and CCSS data without having to do clunky downloads and conversions. Snow depth / SWE / PRCPSA are in meters, temperatures are in celsius. This module is built on my [snotel_ccss_stations](https://github.com/egagli/snotel_ccss_stations) repository, which uses a github action to auto-update the station data daily.**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%aimport easysnowdata

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tqdm
import contextily as ctx

import xarray as xr
import glob

## View all SNOTEL & CCSS stations
- the [SNOwpack TELemetry (SNOTEL) network](https://www.nrcs.usda.gov/wps/portal/wcc/home/aboutUs/monitoringPrograms/automatedSnowMonitoring/) includes over 800 automated weather stations in the Western U.S. for mountain snowpack observation
- the [CCSS program](https://water.ca.gov/Programs/Flood-Management/Flood-Data/Snow-Surveys) manages a network of 130 automated snow sensors located in the Szierra Nevada and Shasta-Trinity Mountains

### Get an up to date GeoDataFrame of all active SNOTEL and CCSS stations

In [16]:
bbox_gdf = gpd.read_file('https://github.com/egagli/sar_snowmelt_timing/raw/main/input/shapefiles/mt_rainier.geojson')

In [17]:
all_stations_gdf = easysnowdata.automatic_weather_stations.get_all_stations(sortby_dist_to_geom=bbox_gdf)
all_stations_gdf

,name,network,elevation_m,latitude,longitude,state,HUC,mgrs,mountainRange,beginDate,endDate,csvData,geometry,dist_km
code,,,,,,,,,,,,,,
679_WA_SNTL,Paradise,SNOTEL,1563.624023,46.782650,-121.747650,Washington,171100150104,10TES,Cascade Range,1979-10-01,2024-03-16,True,POINT (-121.74765 46.78265),1.185449
941_WA_SNTL,Mowich,SNOTEL,963.168030,46.928329,-121.952316,Washington,171100140105,10TES,Cascade Range,1998-09-30,2024-03-15,True,POINT (-121.95232 46.92833),6.070693
1085_WA_SNTL,Cayuse Pass,SNOTEL,1597.151978,46.869541,-121.534302,Washington,171100140301,10TFS,Cascade Range,2006-10-01,2100-01-01,True,POINT (-121.53430 46.86954),8.298225
942_WA_SNTL,Burnt Mountain,SNOTEL,1271.015991,47.044399,-121.940323,Washington,171100140103,10TET,Cascade Range,1999-07-01,2024-03-16,True,POINT (-121.94032 47.04440),11.769572
642_WA_SNTL,Morse Lake,SNOTEL,1648.968018,46.905849,-121.482697,Washington,170300020106,10TFS,Cascade Range,1978-10-01,2024-03-16,True,POINT (-121.48270 46.90585),12.228570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072_AK_SNTL,Kantishna,SNOTEL,472.440002,63.541672,-150.994003,Alaska,190803101805,05VNL,Alaska Range,2004-10-01,2024-03-16,True,POINT (-150.99400 63.54167),2607.639295
1266_AK_SNTL,Telaquana Lake,SNOTEL,388.619995,60.982430,-153.917725,Alaska,190304052303,05VMH,Alaska Range,2014-07-09,2024-03-16,True,POINT (-153.91772 60.98243),2623.640607
958_AK_SNTL,Coldfoot,SNOTEL,316.992004,67.253326,-150.182999,Alaska,190901010604,05WPQ,None,1994-10-01,2024-03-16,True,POINT (-150.18300 67.25333),2814.045738


In [29]:
ds = easysnowdata.automatic_weather_stations.get_all_stations_all_data(all_stations_gdf)

Decompressing data...
Creating xarray.Dataset from the downloaded data...
Removing temporary data...
Done!


<xarray.Dataset> Size: 1GB
Dimensions:        (time: 23824, station: 969)
Coordinates: (12/16)
  * time           (time) object 191kB '1909-04-13' ... '2024-03-15'
  * station        (station) <U12 47kB '660_OR_SNTL' '412_CO_SNTL' ... 'PDS'
    name           (station) <U24 93kB 'New Crescent Lake' ... 'Paradise Meadow'
    network        (station) <U6 23kB 'SNOTEL' 'SNOTEL' ... 'CCSS' 'CCSS'
    elevation_m    (station) float64 8kB 1.497e+03 2.621e+03 ... 2.332e+03
    latitude       (station) float64 8kB 43.51 40.21 37.65 ... 38.48 39.62 38.05
    ...             ...
    mountainRange  (station) object 8kB 'Cascade Range' ... 'Sierra Nevada'
    beginDate      (station) datetime64[ns] 8kB 1979-10-01 ... 2005-10-01
    endDate        (station) datetime64[ns] 8kB 2024-03-16 ... 2024-03-16
    csvData        bool 1B True
    geometry       (station) object 8kB POINT (-121.97982025146484 43.5118484...
    dist_km        (station) float64 8kB 365.3 1.492e+03 ... 801.3 985.7
Data variables:
    TAVG           (station, time) float64 185MB nan nan nan ... -2.8 -3.3 nan
    TMIN           (station, time) float64 185MB nan nan nan ... -11.1 -6.7 nan
    TMAX           (station, time) float64 185MB nan nan nan nan ... 5.0 0.0 nan
    SNWD           (station, time) float64 185MB nan nan nan ... 2.311 2.311
    WTEQ           (station, time) float64 185MB nan nan nan ... 0.8318 0.8331
    PRCPSA         (station, time) float64 185MB nan nan nan nan ... nan nan nan

In [23]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:        (time: 23824, station: 969)
Coordinates: (12/16)
  * time           (time) object 191kB '1909-04-13' ... '2024-03-15'
  * station        (station) <U12 47kB '660_OR_SNTL' '412_CO_SNTL' ... 'PDS'
    name           (station) <U24 93kB 'New Crescent Lake' ... 'Paradise Meadow'
    network        (station) <U6 23kB 'SNOTEL' 'SNOTEL' ... 'CCSS' 'CCSS'
    elevation_m    (station) float64 8kB 1.497e+03 2.621e+03 ... 2.332e+03
    latitude       (station) float64 8kB 43.51 40.21 37.65 ... 38.48 39.62 38.05
    ...             ...
    mountainRange  (station) object 8kB 'Cascade Range' ... 'Sierra Nevada'
    beginDate      (station) datetime64[ns] 8kB 1979-10-01 ... 2005-10-01
    endDate        (station) datetime64[ns] 8kB 2024-03-16 ... 2024-03-16
    csvData        bool 1B True
    geometry       (station) object 8kB POINT (-121.97982025146484 43.5118484...
    dist_km        (station) float64 8kB 365.3 1.492e+03 ... 801.3 985.7
Data variables:
    TAVG           (station, time) float64 185MB nan nan nan ... -2.8 -3.3 nan
    TMIN           (station, time) float64 185MB nan nan nan ... -11.1 -6.7 nan
    TMAX           (station, time) float64 185MB nan nan nan nan ... 5.0 0.0 nan
    SNWD           (station, time) float64 185MB nan nan nan ... 2.311 2.311
    WTEQ           (station, time) float64 185MB nan nan nan ... 0.8318 0.8331
    PRCPSA         (station, time) float64 185MB nan nan nan nan ... nan nan nan

In [30]:
#ds.drop_vars('geometry').to_netcdf(f'all_stations.nc')

### Use geopandas `GeoDataFrame.explore()` on the `all_stations_gdf` geodataframe to interactively view the stations 
- color by network: red is SNOTEL, blue is CCSS.

In [ ]:
all_stations_gdf.astype(dict(beginDate=str, endDate=str)).explore(column='network',cmap='bwr')

## Get data for a singular site: *Is our winter on track with the historical record at the Paradise, WA SNOTEL station?*
- check out information about the [SNOTEL station near Mt. Rainier at Paradise, WA](https://wcc.sc.egov.usda.gov/nwcc/site?sitenum=679)
- cool plots available at the [Northwest River Forecast Center website](https://www.nwrfc.noaa.gov/snow/snowplot.cgi?AFSW1)

### Place a station code (which you can find in this interactive plot, or by other means) in the url: https://raw.githubusercontent.com/egagli/snotel_ccss_stations/main/data/{station_id}.csv
- for SNOTEL stations, this will be of the form {unique number}_{two letter state abbreviation}_SNTL (e.g. 679_WA_SNTL).   
- for CCSS stations, this will be a three letter code (e.g. BLK).   
- use `pd.read_csv()` with `index_col='datetime'` and `parse_dates=True` so we interpret the datetime column as pandas datetime objects

In [ ]:
station_id = '679_WA_SNTL'
paradise_snotel = pd.read_csv(f'https://raw.githubusercontent.com/egagli/snotel_ccss_stations/main/data/{station_id}.csv',index_col='datetime', parse_dates=True)

In [ ]:
paradise_snotel

### Try a simple plot of snow depth and SWE
- select the column of interest and use pandas built in `Series.plot()`

In [ ]:
f,ax=plt.subplots(figsize=(12,5))

paradise_snotel['SNWD'].plot(ax=ax,label='snow depth')
paradise_snotel['WTEQ'].plot(ax=ax,label='snow water equivalent')

ax.set_xlim(pd.to_datetime(['2017-10-01','2018-09-30']))

ax.grid()
ax.legend()

ax.set_xlabel('time')
ax.set_ylabel('snow depth / SWE [meters]')
ax.set_title('Snow depth and SWE at Paradise, WA \n(water year 2018)')

f.tight_layout()

### Try a more complex plot that shows current snow depth against statistics calculated from the entire time series for each day of water year
- water year is conceptual 12 month period used to describe when the bulk of precipitation falls, mostly used for hydrology attribution 
    - in the northern hemisphere, we usually define the water year to start October 1st and go until September 30th (e.g. water year 2017: October 1st, 2016 - September 30th, 2017)
    - so October 1st is DOWY 1
- try a function like `datetime_to_DOWY()` shown below to convert datetimes to day of water year and add a dedicated DOWY column
    - this function should account for leap years
- then use pandas groupby functionality to calculate stats per DOWY
- plot these stats
    - thanks David Shean for the plot inspiration!

In [ ]:
def datetime_to_DOWY(date):
    try:
        if date.month < 10 or (date.month == 10 and date.day < 1):
            start_of_water_year = pd.Timestamp(year=date.year-1, month=10, day=1)
        else:
            start_of_water_year = pd.Timestamp(year=date.year, month=10, day=1)
        return (date - start_of_water_year).days + 1
    except:
        return np.nan

def datetime_to_WY(date):
    if date.month < 10 or (date.month == 10 and date.day < 1):
        return date.year
    else:
        return date.year + 1

In [ ]:
paradise_snotel['DOWY'] = paradise_snotel.index.map(datetime_to_DOWY)

In [ ]:
stat_list = ['min','max','mean','std','median']
paradise_snotel_DOWY_snwd_stats = paradise_snotel.groupby('DOWY').agg(stat_list)['SNWD']
paradise_snotel_DOWY_snwd_stats

In [ ]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
current_WY = slice(f'{int(today[0:4])-1}-10-01',f'{today}')
current_WY_paradise_snotel = paradise_snotel[current_WY.start:current_WY.stop]

In [ ]:
f,ax=plt.subplots(figsize=(12,7))

for stat,stat_color in zip(['min','max','mean','median'],['red','blue','mediumpurple','mediumseagreen']):
    ax.plot(paradise_snotel_DOWY_snwd_stats.index, paradise_snotel_DOWY_snwd_stats[stat], label=stat, color=stat_color, linewidth=3)
    
ax.fill_between(paradise_snotel_DOWY_snwd_stats.index, paradise_snotel_DOWY_snwd_stats['mean'] - paradise_snotel_DOWY_snwd_stats['std'], paradise_snotel_DOWY_snwd_stats['mean'] + paradise_snotel_DOWY_snwd_stats['std'], color='mediumpurple', alpha=0.3, label='mean +/- 1 std')

ax.scatter(current_WY_paradise_snotel.DOWY,current_WY_paradise_snotel.SNWD, marker='o', color= 'black', label='Current WY')

ax.set_xlim([0,366])
ax.set_ylim([0,6])

ax.grid()
ax.legend()

ax.set_title(f'Current snow depth against historical snow depth stats by DOWY at Paradise, WA\n({paradise_snotel.index.min().date()} - {paradise_snotel.index.max().date()})')
ax.set_xlabel('Day of Water Year [Oct 1 - Sept 30]')
ax.set_ylabel('Snow depth [meters]')
f.tight_layout()

**Looks like we're below the mean for snow depth for today's DOWY.**

## Read a variable from multiple CSVs by looping over a list of stations: *Does the SNOTEL network and CCSS network list the same station?*
- no controversy here, but i noticed that a station name was repeated
- perhaps both networks have data from the same station accessible
- might make sense if they are co-managed in some way?

### Create a list of the stations we are interested in, loop through and add data to a dictionary with the station code as the key, then read into pandas using `pd.DataFrame.from_dict()`
- initialize with an empty dict and append to it in the loop

In [ ]:
station_list = ['356_CA_SNTL','BLK']

station_dict = {}

for station in station_list:
    tmp = pd.read_csv(f'https://raw.githubusercontent.com/egagli/snotel_ccss_stations/main/data/{station}.csv',index_col='datetime',parse_dates=True)['WTEQ']
    station_dict[station] = tmp

stations_swe_df = pd.DataFrame.from_dict(station_dict)#.dropna()

In [ ]:
stations_swe_df

### Plot the two stations on the same axis
- if we properly set the index as the datetime, and we used `parse_dates=True`, these should line up!

In [ ]:
f,ax=plt.subplots(figsize=(20,7))

stations_swe_df.plot(ax=ax,color=['red','blue'])

ax.legend()

ax.set_xlabel('time')
ax.set_ylabel('snow water equivalent [meters]')
ax.set_title('SNOTEL and CCSS SWE at Blue Lakes, CA \nmaybe these are the same? :)')

f.tight_layout()

#ax.set_xlim(['2018-08-01','2020-01-01'])

### These look oddly similar... let's check out their correlation
- convenient built in `DataFrame.corr()`

In [ ]:
stations_swe_df.corr()

**These correlation values, along with the time series above, makes me think these are way too similar... no way these would agree this much even if the stations were right next to each other!**

### Let's see where they exist spatially
- select the stations by index and reproject to UTM 11N
- use `contextily` for a basemap

In [ ]:
f,ax=plt.subplots(figsize=(7,7))

all_stations_gdf[all_stations_gdf.index=='356_CA_SNTL'].to_crs('EPSG:32611').plot(ax=ax, color='red',label='356_CA_SNTL')
all_stations_gdf[all_stations_gdf.index=='BLK'].to_crs('EPSG:32611').plot(ax=ax, color='blue',label='BLK')

ax.set_xlim([244200,245700])
ax.set_ylim([4276900,4278700])

ctx.add_basemap(ax, crs='EPSG:32611', source=ctx.providers.Esri.WorldImagery)

ax.legend()

f.tight_layout()

**Interesting locations :) Based on correlation and location, I'm going to say these are the same! Wonder what those tiny differences are about...**

### Look's like some of the CCSS stations have "Natural Resources Conservation Service" as their operator
- Let's check which stations these are!
- Let's plot these shared stations in red, and not shared in blue

In [ ]:
!pip install -q lxml
import requests 
from io import StringIO

csv = 'https://cdec.water.ca.gov/misc/SnowSensors.html'
response = requests.get(csv)
same_stations_df = pd.read_html(StringIO(response.content.decode('utf-8')))[0].set_index('ID').sort_index()
same_stations_df = same_stations_df[same_stations_df.nunique(axis=1) > 1]
same_stations_gdf = gpd.GeoDataFrame(same_stations_df, geometry=gpd.points_from_xy(same_stations_df['Longitude'], same_stations_df['Latitude']))
same_stations_gdf.crs = "EPSG:4326"
same_stations_gdf = same_stations_gdf[same_stations_gdf['Operator Agency']=='Natural Resources Conservation Service']
same_stations_gdf

In [ ]:
f,ax=plt.subplots(figsize=(10,10))

same_stations_gdf.to_crs("EPSG:32611").plot(ax=ax,color='red',label='CCSS station operated by NRCS')
ccss_stations_gdf = all_stations_gdf[all_stations_gdf['network']=='CCSS'].to_crs("EPSG:32611")
ccss_stations_gdf[~ccss_stations_gdf.index.isin(same_stations_gdf.index)].plot(ax=ax,color='blue',label='CCSS station not operated by NRCS')

ctx.add_basemap(ax, crs='EPSG:32611', source=ctx.providers.Esri.WorldImagery)

ax.legend()

f.tight_layout()

**There are 33 of these! These are likely also listed as SNOTEL stations...**

## Read a variable from multiple CSVs by looping over a subset of the geodataframe: *How extraordinary was the California 2023 snowpack?*
- the Sierra Nevada [received a historic amount of snow in 2023](https://www.nps.gov/articles/000/sien-sierranevadamonitor-spring2023.htm)
- let's explore the magnitude of this season by comparing to the median snow pack

### As before, create a list of the stations we are interested in, loop through and add data to a dictionary with the station code as the key, then read into pandas using `pd.DataFrame.from_dict()`
- create a geodataframe `ccss_stations_gdf` of only CCSS stations from `all_stations_gdf` by creating an index where network equals CCSS
- loop through the CCSS stations and create a dataframe `ccss_stations_snwd_df`

In [ ]:
ccss_stations_gdf = all_stations_gdf[all_stations_gdf['network']=='CCSS']

In [ ]:
ccss_stations_gdf

In [ ]:
%%time 
station_dict = {}

for station in tqdm.tqdm(ccss_stations_gdf.index):
    try:
        tmp = pd.read_csv(f'https://raw.githubusercontent.com/egagli/snotel_ccss_stations/main/data/{station}.csv',index_col='datetime',parse_dates=True)['SNWD']
        station_dict[station] = tmp
    except:
        print(f'failed to retrieve {station}')

ccss_stations_snwd_df = pd.DataFrame.from_dict(station_dict).dropna(how='all')

In [ ]:
ccss_stations_snwd_df

### Let's check out the percent of normal snow depth for April 1st, 2023
- let's add a DOWY column to `ccss_stations_snwd_df` like before, groupby DOWY, apply a median, and divide the observation on April 1st, 2023 by the DOWY 183 (April 1) median
    - add these percent normal values back to `ccss_stations_gdf`
- will need to do some slight cleaning to get rid of NaNs, Infs, physically impossible values...

In [ ]:
ccss_stations_snwd_df['DOWY'] = ccss_stations_snwd_df.index.map(datetime_to_DOWY)

In [ ]:
ccss_stations_gdf.loc[:,'april2023_percent_norm'] = 100*(ccss_stations_snwd_df['2023-04-01':'2023-04-01'].squeeze() / ccss_stations_snwd_df.groupby('DOWY').median().loc[183])

In [ ]:
ccss_stations_gdf = ccss_stations_gdf.dropna(subset='april2023_percent_norm')
ccss_stations_gdf = ccss_stations_gdf[ccss_stations_gdf['april2023_percent_norm']<7500]

### View the values in order...
- use `.sort_values()` function to get an idea of percent normal snow depth values
- `DataFrame.head()` and `DataFrame.tail()` to see highest and lowest percent normal snow depth values

In [ ]:
ccss_stations_gdf.sort_values('april2023_percent_norm',ascending=False).head()

In [ ]:
ccss_stations_gdf.sort_values('april2023_percent_norm',ascending=False).tail()

### Plot the percent of normal snow depths for April 1st, 2023
- add elevation plot to the right with horizontal dashed line at 100% (normal)

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,9),gridspec_kw={'width_ratios': [2, 1]})

ccss_stations_gdf.to_crs('EPSG:32611').plot(ax=ax[0], column='april2023_percent_norm',legend=True,vmin=0,vmax=500,cmap='gnuplot',edgecolor='black',s=100)

ctx.add_basemap(ax[0], crs='EPSG:32611', source=ctx.providers.Esri.WorldImagery, attribution='')

ax[0].set_title('station locations')


ax[1].scatter(ccss_stations_gdf.elevation_m,ccss_stations_gdf.april2023_percent_norm,c=ccss_stations_gdf.april2023_percent_norm,cmap='gnuplot',vmin=0,vmax=500,edgecolors='black',s=100)
ax[1].axhline(y=100,linestyle='--',color='black')

ax[1].set_title('station percent normal snow depth vs elevation')
ax[1].set_xlabel('elevation [m]')
ax[1].set_ylabel('percent of normal snow depth [%]')

f.suptitle(f'California percent normal snow depth for April 1st, 2023')
f.tight_layout()

**Looks like a lot more snow than usual to me!**

## Read a variable from all CSVs by looping over the entire geodataframe: *Has the date of maximum SWE changed in the Western US?*
- [Snowmelt timing can be an important indicator of regional climate change](https://www.epa.gov/climate-indicators/climate-change-indicators-snowpack), and the snowmelt timing of the Western U.S. is projected to shift earlier in the year by up to 1 month by 2050 ([Barnett et al., 2005](https://www.nature.com/articles/nature04141); [Stewart, 2009](https://onlinelibrary.wiley.com/doi/10.1002/hyp.7128)), with a corresponding snowpack loss equivalent to a 25% decrease in streamflow from snowmelt ([Siirila-Woodburn et al., 2021](https://www.nature.com/articles/s43017-021-00219-y)).  
- Let's explore trends in maximum SWE timing using SNOTEL and CCSS stations

### This time loop through all stations and and add data to a dictionary with the station code as the key, then read into pandas using `pd.DataFrame.from_dict()`
- might take a minute to load in almost 1000 CSVs...
- store SWE for all stations in `all_stations_swe_df`

In [ ]:
%%time 
station_dict = {}

for station in tqdm.tqdm(all_stations_gdf.index):
    try:
        tmp = pd.read_csv(f'https://raw.githubusercontent.com/egagli/snotel_ccss_stations/main/data/{station}.csv',index_col='datetime',parse_dates=True)['WTEQ']
        station_dict[station] = tmp
    except:
        print(f'failed to retrieve {station}')

In [ ]:
all_stations_swe_df = pd.DataFrame.from_dict(station_dict)
all_stations_swe_df

### Prepare the data
- prepare and clean `all_stations_swe_df`:
    - filter to start in WY 1967 (the first year with more than one station) and end with WY 2023
    - add water year column
    - remove any negative SWE measurements
    - for consistency with similar analyses, following the methodology of [Evan 2019](https://journals.ametsoc.org/view/journals/apme/58/1/jamc-d-18-0150.1.xml) and [US EPA 2021](https://www.epa.gov/sites/default/files/2021-04/documents/snowpack_td.pdf):
       - set any change of greater magnitude than 20cm to NaN
       - if there are more than 30 days of missing data during November-April, don't use that water year
       - if SWE is zero during every day of Jan/Feb/March, don't use that water year
       - only use stations with continuous data from WY 1982 
           - i do this only for the all data bulk calculation so we have a common reference frame, but for station level analysis i instead impose a 30 year or longer record rule

In [ ]:
all_stations_swe_df = all_stations_swe_df.loc[slice('1966-10-01','2023-09-30')]

In [ ]:
all_stations_swe_df['WY'] = all_stations_swe_df.index.map(datetime_to_WY)

In [ ]:
all_stations_swe_df = all_stations_swe_df[all_stations_swe_df>=0] 

In [ ]:
all_stations_swe_diff_df = all_stations_swe_df.diff().abs()
all_stations_swe_df[all_stations_swe_diff_df>0.20] = np.nan

In [ ]:
def check_missing_data(group):
    nov_to_apr_mask = group.index.month.isin([11, 12, 1, 2, 3, 4])
    filtered_group = group[nov_to_apr_mask]
    missing_data_counts = filtered_group.isnull().sum()
    columns_to_nan = missing_data_counts[missing_data_counts > 30].index
    group[columns_to_nan] = np.nan
    return group

def check_zero_swe(group):
    for month in [1, 2, 3]:
        month_mask = group.index.month == month
        zero_swe_columns = group[month_mask].eq(0).all()
        columns_to_nan = zero_swe_columns[zero_swe_columns].index
        group[columns_to_nan] = np.nan
    return group

In [ ]:
all_stations_swe_df = all_stations_swe_df.groupby('WY').apply(check_missing_data).droplevel(0)
all_stations_swe_df = all_stations_swe_df.groupby('WY').apply(check_zero_swe).droplevel(0)
all_stations_swe_df

### Calculate and plot trend in DOWY of max SWE for all data 
- use `DataFrame.groupby()` to find the date of max SWE per year and convert to a DOWY value, store in `all_stations_dowy_max_swe_df`
- calculate slope and intercept of linear regression using `np.polyfit()` on a melted version of  
- obtain statistics for each year using `DataFrame.describe()`

In [ ]:
all_stations_dowy_max_swe_df = all_stations_swe_df.groupby('WY').idxmax().applymap(datetime_to_DOWY)

In [ ]:
stations_before_WY1982 = all_stations_gdf[all_stations_gdf.beginDate<'1981-10-01']
dowy_max_swe_melted = pd.melt(all_stations_dowy_max_swe_df.reset_index(),id_vars='WY').dropna()
dowy_max_swe_melted_before_WY1982 = dowy_max_swe_melted[dowy_max_swe_melted['variable'].isin(stations_before_WY1982.index)]
slope, intercept = np.polyfit(dowy_max_swe_melted_before_WY1982.WY,dowy_max_swe_melted_before_WY1982.value,1)
lr_years = np.unique(dowy_max_swe_melted.WY)

In [ ]:
describe = all_stations_dowy_max_swe_df.T.describe()
describe

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,6),sharex=True,gridspec_kw={'height_ratios': [3, 2]})

describe.loc['50%'].plot(ax=ax[0],label='median')

ax[0].fill_between(describe.columns,describe.loc['25%'],describe.loc['75%'],alpha=0.3,label='IQR')
ax[0].plot(lr_years,np.array(lr_years)*slope+intercept,'k--',label=f'Trend (slope={slope:.2f} Days/Year)')
#ax[0].set_xlim([1967,2023])

ax[0].legend()

describe.loc['count'].plot(ax=ax[1])


ax[0].set_title('Trend in DOWY of max SWE')
ax[1].set_title('Number of active stations')

### Check out trend at each station seperately
- calculate the linear trend in DOWY of max SWE only for stations with over 30 years of data, store in our original `all_stations_gdf`
    - out of 971 stations with SWE data, 645 meet this criteria
- plot the trend for each station and plot the trends on a histogram

In [ ]:
all_stations_gdf.loc[:,'dowy_max_swe_trend'] = all_stations_dowy_max_swe_df.apply(lambda y: np.polyfit(y.dropna().index.values, y.dropna(), 1)[0] if len(y.dropna()) >= 30 else np.nan)

In [ ]:
f,ax=plt.subplots(figsize=(10,5.5))

all_stations_gdf.plot(column='dowy_max_swe_trend',ax=ax,legend=True,cmap='RdBu_r',edgecolor='k',markersize=20,vmin=-1,vmax=1,legend_kwds={'label':'[days/year]\n(Red is later in the year, blue is earlier)'})

ctx.add_basemap(ax, crs='EPSG:4326', source=ctx.providers.Esri.WorldImagery, attribution='')

ax.set_title('Trend in DOWY of max SWE\n(only stations with 30+ years of data)')

In [ ]:
f,ax=plt.subplots()

ax.hist(all_stations_gdf['dowy_max_swe_trend'],bins=50)

ax.axvline(x=0,color='red')

ax.set_xlim([-1.5,1.5])

ax.set_xlabel('trend [days/year]')
ax.set_ylabel('count')
ax.set_title('Distribution of trends in DOWY of max SWE')

### Let's analyze these trends by mountain range
- we can `DataFrame.groupby()` our geodataframe by mountain range to calculate mountain range specific stats, store in `mountain_range_trend_df`
- mountain ranges with more stations (and more spatial coverage) are probably more trustworthy

In [ ]:
mountain_range_count = all_stations_gdf.dropna().groupby('mountainRange')['dowy_max_swe_trend'].count()
mountain_range_median = all_stations_gdf.dropna().groupby('mountainRange')['dowy_max_swe_trend'].median()
mountain_range_mean = all_stations_gdf.dropna().groupby('mountainRange')['dowy_max_swe_trend'].mean()
mountain_range_std = all_stations_gdf.dropna().groupby('mountainRange')['dowy_max_swe_trend'].std()

In [ ]:
mountain_range_trend_df = pd.concat([mountain_range_count,mountain_range_median,mountain_range_mean,mountain_range_std],axis=1)
mountain_range_trend_df.columns = ['station_count','median','mean','std']

In [ ]:
mountain_range_trend_df

In [ ]:
f,ax=plt.subplots(1,2,sharey=True,gridspec_kw={'width_ratios': [1, 3]})

mountain_range_trend_df['station_count'].plot.barh(ax=ax[0])
mountain_range_trend_df['median'].plot.barh(ax=ax[1],cmap='RdBu')


ax[0].set_xlabel('[#]')
ax[1].set_xlabel('[days/year]')
ax[0].set_ylabel('')
ax[0].set_title('station count')
ax[1].set_title('trend')

f.suptitle('Trend in DOWY of max SWE by mountain range')

### Let's visualize this on a map 
- add spatial information to `mountain_range_trend_df` using `DataFrame.join()` with mountain range geometries from [GMBA Mountain Inventory v2](https://www.earthenv.org/mountains)
- ceate both a static plot with counts and medians and create an interactive plot so we can explore the trends across mountain ranges and stations

In [ ]:
url = f'https://data.earthenv.org/mountains/standard/GMBA_Inventory_v2.0_standard_300.zip'
gmba_gdf = gpd.read_file('zip+'+url)

In [ ]:
mountain_range_trend_gdf = gpd.GeoDataFrame(mountain_range_trend_df.join(gmba_gdf[['MapName','geometry']].set_index('MapName')))

In [ ]:
f,ax=plt.subplots(1,2,figsize=(14,7))

mountain_range_trend_gdf.plot(ax=ax[0],column='station_count',vmin=0,vmax=100,cmap='viridis',legend=True,edgecolor='k',legend_kwds={'label':'[#]'})
mountain_range_trend_gdf.plot(ax=ax[1],column='median',vmin=-0.3,vmax=0.3,cmap='RdBu_r',legend=True,edgecolor='k',legend_kwds={'label':'[days/year]\n(Red is later in the year, blue is earlier)'})

for axs in ax:
    ctx.add_basemap(ax=axs, crs=mountain_range_trend_gdf.crs, source=ctx.providers.Esri.WorldImagery, attribution=False)
    ctx.add_basemap(ax=axs, crs=mountain_range_trend_gdf.crs, source=ctx.providers.Esri.WorldImagery, attribution=False)
    axs.set_xlim([-125,-104])
    axs.set_ylim([27,55])
    
ax[0].set_title('count')
ax[1].set_title('trend')

f.suptitle('Trend in DOWY of max SWE by mountain range\n(only stations with 30+ years of data)')

In [ ]:
m = mountain_range_trend_gdf.explore(column='median',cmap='RdBu_r',vmin=-0.5,vmax=0.5)
all_stations_gdf.astype(dict(beginDate=str, endDate=str)).explore(m=m,column='dowy_max_swe_trend',cmap='RdBu_r',vmin=-0.3,vmax=0.3)

**Looks like these trends are different by region, but relatively consistent within region. The majority of regions show the timing of maximum SWE happening earlier in the year, with notable exceptions being mountain ranges in the Pacific Northwest which show a reverse trend with smaller magnitude. Also of importance is the number of stations and their spatial dsitribution in each region, as 2 of the 4 regions (Olympic Mountains and Oregon Coast Range) showing the timing of maximum SWE happening later in the year only have one station each with a 30+ year record.**